##### Resources
- [https://arxiv.org/abs/0811.3171]("Quantum algorithm for solving linear systems of equations")
- [https://qiskit.org/textbook/ch-applications/hhl_tutorial.html](Qiskit Notebook)
- [https://homepages.cwi.nl/~rdewolf/qcnotes.pdf](Notes on Quantum Computing)
- [https://quantumalgorithmzoo.org/](Quantum Algorithm Zoo)

#### HHL Algorithm

Data una matrice $A\in \mathbb{C}^{N\times N}$ e un vettore $\overrightarrow{b}\in\mathbb{C}^{N}$ trovare $\overrightarrow{x}\in\mathbb{C}^{N}$ che soddisfa $A\overrightarrow{x} = \overrightarrow{b}$

Un sistema di equazioni lineari è detto $s$-sparso se $A$ ha al più $s$ elementi $\neq 0$ per colonna o riga.

Dato $k$ numero di condizionamento del sistema e $\epsilon$ accuratezza, risolvere un sistema $s$-sparso di dimensione $N$ su un computer classico richiede $O(Nsk\log\left(\frac{1}{\epsilon}\right))$

L'algoritmo HHL richiede tempo $O(\frac{\log(N)s^2k^2}{\epsilon})$ per stimare una funzione della soluzione quando $A$ è una matrice Hermitiana, sotto l'assunzione di avere oracoli efficienti per caricare i dati, simulatzione Hamiltoniana. Lo speedup è esponenziale nella dimensione del sistema, con una differenza importante: l'algoritmo classico ritorna la soluzione completa, mentre HHL può solo approssimare funzioni del vettore soluzione.

##### Codificare il problema nel mondo quantico

Riscalando il sistema, possiamo assumere $\overrightarrow{b}$ e $\overrightarrow{x}$ normalizzati e mapparli negli stati quantici rispettivi $\ket{b}$ e $\ket{x}$. Solitamente il mapping è tale che l'$i$-esimo componente di $\overrightarrow{b}$ corrisponde all'ampiezza dell'$i$-esimo basis state dello stato quantico $\ket{b}$, idem per $\overrightarrow{x}$ e $\ket{x}$

Da qui in poi consideriamo il problema riscalato $$A\ket{x} = \ket{b}$$
Dato che $A$ è Hermitiana, che significa che è una matrice quadrata complessa uguale alla sua coniugata trasposta (cioè $a_{ij} = \overline{a_ij} \Leftrightarrow A = \overline{A^T} \Leftrightarrow A = A^H$), allora ha una decomposizione spettrale $$A=\sum_{j=0}^{N-1} \lambda_j\ket{u_j}\bra{u_j}$$ con $\lambda_j\in\mathbb{R}$ e $\ket{u_j}$ il $j$-esimo autovettore di $A$ rispetto all'autovalore $\lambda_j$. Questo porta a $$A^{-1} = \sum_{j=0}^{N-1}\lambda_j^{-1}\ket{u_j}\bra{u_j}$$ Quindi la parte destra del sistema può essere riscritta in termini della base di autovettori di $A$ come $$\ket{b} = \sum_{j=1}^{N-1}b_j\ket{u_j}$$ con $b_j\in\mathbb{C}$

Ricordare che l'obiettivo di HHL è terminare l'algoritmo con il registro di readout nello stato $$\ket{x} = A^{-1}\ket{b} = \sum_{j=0}^{N-1}\lambda_j^{-1}b_j\ket{u_j}$$

##### Descrizione dell'Algoritmo HHL

L'algoritmo usa 3 registri quantici, tutti settati a $\ket{0}$ all'inizio.
- $n_l$ è usato per memorizzare la rappresentazione binaria degli autovalori di $A$
- $n_b$ contiene il vettore soluzione ($N = 2^{n_b}$)
- Un registro extra per i qubit ausiliari, usati come passaggi intermedi nelle computazioni singole, ignorati perché settati a $\ket{0}$ all'inizio e risettati a $\ket{0}$ alla fine della singola operazione

![circuit](img/hhlcircuit.png)

1. Carica $\ket{b}\in\mathbb{C}^N$, cioè esegue la trasformazione $\ket{0}_{n_b}\mapsto\ket{b}_{n_b}$
2. Applica QPE (Quantum Phase Estimation) con $$U = e^{iAt} = \sum_{j=0}^{N-1}e^{i\lambda_j t}\ket{u_j}\bra{u_j}$$
Lo stato quantim del resgistro espresso nella base di autovalori di $A$ è ora $$\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}$$ con $\ket{\lambda_j}_{n_l}$ la rappresentazione binaria di $lambda_j$ su $n_l$ bit
3. Aggiungi un qubit ausiliari e applica la rotazione condizionata su $\ket{\lambda_j}$ $$\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}\left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\ket{0}+\frac{C}{\lambda_j}\ket{1}\right)$$ dove $C$ è una costante di normalizzazione e, espressa come sopra, dovrebbe essere $|C| < \lambda_{\min}$
4. Applica QPE $^T$, e ignorando i possibili errori risulterà  in $$\sum_{j=0}^{N-1}b_j\ket{0}_{n_l}\ket{u_j}_{n_b}\left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\ket{0}+\frac{C}{\lambda_j}\ket{1}\right)$$ 
5. Misura il bit ausiliario nella base computazionale. Se è 1, allora il registro è nello stato di post-misurazione $$\left(\sqrt{\frac{1}{\sum_{j=0}^{N-1}\frac{|b_j|^2}{|\lambda_j|^2}}}\right)\sum_{j=0}^{N-1}\frac{b_j}{\lambda_j}\ket{0}_{n_1}\ket{u_j}_{n_b}$$ corrispondente a soluzione a meno di un fattore di normalizzazione.
6. Applica un osservabile $M$ per calcolare $F(x) = \langle x\:|\:M\:|\:x\rangle$

##### QPE in HHL

QPE è un algoritmo quantico che, data una matrice unitaria $U$ con autovettore $\ket{\psi}$ e autovalore $e^{2\pi i\theta}$, trova $\theta$. La definizione formale è quella che segue:

Data $U\in\mathbb{C}^{2^m\times 2^m}$ unitaria e $\ket{\psi}_m\in \mathbb{C}^{2^m}$ uno dei suoi autovettori di autovalore $e^{2\pi i\theta}$, QPE prende in input $U$ e lo stato $\ket{0}_n\ket{\psi}_m$ e ritorna lo stato $\ket{\overline{\theta}}_n\ket{\psi}_m$ con $\overline{\theta}$ approssimazione binaria di $2^n\theta$ e $_n$ a indicare il troncamento a $n$ cifre.
$$QPE(U,\ket{0}_n\ket{\psi}_m)=\ket{\overline{\theta}}_n\ket{\psi}_m$$

Per HHL useremo QPE con $U = e^{iAt}$, dove $A$ è la matrice associata al sistema che vogliamo risolvere. In questo caso:
$$e^{iAt}=\sum_{j=0}^{N-1}e^{i\lambda_j t}\ket{u_j}\bra{u_j}$$

Per l'autovettore $\ket{u_j}_{n_b}$, che ha autovalore $e^{i\lambda_j t}$, QPE emetterà in output $\ket{\overline{\lambda}_j}_{n_l}\ket{u_j}_{n_b}$. $\overline{\lambda}_j$ rappresenta una rappresentazione binaria su $n_l$ bit di $2^{n_l}\frac{\lambda_jt}{2\pi}$. Quindi, se ogni $\lambda_j$ può essere rappresentato esattamente su $n_l$ bits, abbiamo
$$QPE\left(e^{iAt},\sum_{j=0}^{N-1}b_j\ket{0}_{n_l}\ket{u_j}_{n_b}\right)=\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}$$

In realtà, lo stato quantico nel registro dopo aver applicato QPE allo stato iniziale è $$\sum_{j=0}^{N-1}b_j\left(\sum_{l=0}^{2^{n_l}-1}\alpha_{l|j}\ket{l}_{n_l}\right)\ket{u_j}_{n_b}$$
con $$\alpha_{l|j} = \frac{1}{2^{n_l}}\sum_{k=0}^{2^{n_l}-1}\left(e^{2\pi i\left(\frac{\lambda_j t}{2\pi}-\frac{l}{2^{n_l}}\right)}\right)^k$$
Denotiamo con $\overline{\lambda_j}$ la migliore approssimazione su $n_l$ bit di $\lambda_j$ con $1\leq j\leq N$, possiamo ridefinire il registro $n_l$ così che $\alpha_{l|j}$ denota l'ampiezza di $\ket{l+\overline{\lambda}_j}_{n_l}$
$$\alpha_{l|j} = \frac{1}{2^{n_l}} \sum_{k=0}^{2^{n_l}-1} \left(e^{2\pi i\left(\frac{\lambda_j t}{2\pi}-\frac{l+\overline{\lambda}_j}{2^{n_l}}\right)}\right)^k$$

Se ogni $\frac{\lambda_jt}{2\pi}$ può essere rappresentato esattamente su $n_l$ bits, allora $\frac{\lambda_jt}{2\pi} = \frac{\overline{\lambda}_j}{2^{n_l}}$ for each $j$. Quindi in questo caso $\forall\:j\:\:1\leq j\leq N$ abbiamo che $\alpha_{0|j}=1$ e $\alpha_{l|j} = 0\:\:\forall\:l\neq 0$, solo in questo caso possiamo scrivere che lo stato del registro dopo QPE è $$\sum_{j=0}^{N-1}b_j\ket{\lambda_j}_{n_l}\ket{u_j}_{n_b}$$
Altrimenti $|\alpha_{l|j}|$ è grande $\Leftrightarrow \frac{\lambda_jt}{2\pi} \simeq \frac{l+\overline{\lambda}_j}{2^{n_l}}$ e lo stato del registro è
$$\sum_{j=0}^{N-1}\sum_{l=0}^{2^{n_l}-1}\alpha_{l|j}b_j\ket{l}_{n_l}\ket{u_j}_{n_b}$$

##### Esempio con 4 qubits

$$A=\left(\begin{array}{cc}1&-\frac{1}{3}\\-\frac{1}{3}&1\end{array}\right)\:\:\ket{b}=\left(\begin{array}{c}1\\0\end{array}\right)$$

Useremo $n_b=1$ per rappresentare $\ket{b}$ e la soluzione $\ket{x}$, $n_l=2$ qubits per memorizzare la rappresentazione binaria degli autovalori a $1$ qubit ausiliario per memorizzare se la rotazione condizionata, e quindi l'algoritmo, ha avuto successo o no.

Per poter mostrare meglio l'algoritmo calcoliamo gli autovalori di $A$ in modo da scegliere $t$ così da ottenere una rappresentazione binaria esatta degli autovalori riscalati nel registro $n_l$. Notare che nell'implementazione dell'algoritmo HHL non si ha bisogno di conoscenza pregressa sugli autovalori.
$$\lambda_1=\frac{2}{3}\:\:\lambda_2=\frac{4}{3}$$
QPE emetterà un'approssimazione binaria su $n_l$ bit ($2$ in questo esempio) di $\frac{\lambda_jt}{2\pi}$, quindi impostando $t=2\pi\cdot\frac{3}{8}$ QPE darà un'approssimazione binaria su $2$ bit di $$\frac{\lambda_1t}{2\pi}=\frac{1}{4}\:\:\frac{\lambda_2t}{2\pi}=\frac{1}{2}$$
che sono rispettivamente $$\ket{01}_{n_1}\:\:\ket{10}_{n_l}$$
dai rispettivi autovettori $$\ket{u_1}\frac{1}{\sqrt{2}}\left(\begin{array}{c}1\\-1\end{array}\right)\:\:\ket{u_2}\frac{1}{\sqrt{2}}\left(\begin{array}{c}1\\1\end{array}\right)$$
Una generica matrice Hermitiana $A$ di dimensione $N$ può avere fino a $N$ autovalori distinti, richiedendo così $O(N)$ per il calcolo e il vantaggio quantistico verrebbe perso.

Possiamo riscrivere $\ket{b}$ in termini della base di autovalori di $A$ come $$\ket{b}_{n_b} = \sum_{j=1}^2\frac{1}{\sqrt{2}}\ket{u_j}_{n_b}$$

Ora vediamo i diversi passaggi dell'algoritmo:

1. La preparazione dello stato è triviale, dato che $\ket{b} = \ket{0}$
2. Applicare QPE darà $$\frac{1}{\sqrt{2}}\ket{01}\ket{u_1} + \frac{1}{\sqrt{2}}\ket{10}\ket{u_2}$$
3. Rotazione condizionata con $C=\frac{1}{8}$ che è minore dell'autovalore minimo (riscalato) pari a $\frac{1}{4}$. Notare che la costante $C$ qui deve essere scelta in modo che sia minore dell'autovalore minimo (riscalato) ma il più grande possibile così che quando il qubit ausiliario è misurato la probabilità di essere nello stato $\ket{1}$ è grande.
$$\frac{1}{\sqrt{2}}\ket{01}\ket{u_1}\left(\sqrt{1-\frac{\left(\frac{1}{8}\right)^2}{\left(\frac{1}{4}\right)^2}}\ket{0} + \frac{\frac{1}{8}}{\frac{1}{4}}\ket{1}\right) + \frac{1}{\sqrt{2}}\ket{10}\ket{u_2}\left(\sqrt{1-\frac{\left(\frac{1}{8}\right)^2}{\left(\frac{1}{2}\right)^2}}\ket{0} + \frac{\frac{1}{8}}{\frac{1}{2}}\ket{1}\right) =$$ 
$$= \frac{1}{\sqrt{2}}\ket{01}\ket{u_1}\left(\sqrt{1-\frac{1}{4}}\ket{0} + \frac{1}{2}\ket{1}\right) + \frac{1}{\sqrt{2}}\ket{10}\ket{u_2}\left(\sqrt{1-\frac{1}{16}}\ket{0} + \frac{1}{4}\ket{1}\right)$$
4. Dopo aver applicato QPE $^T$ il quantum computer è nello stato
$$\frac{1}{\sqrt{2}}\ket{00}\ket{u_1}\left(\sqrt{1-\frac{1}{4}}\ket{0} + \frac{1}{2}\ket{1}\right) + \frac{1}{\sqrt{2}}\ket{00}\ket{u_2}\left(\sqrt{1-\frac{1}{16}}\ket{0} + \frac{1}{4}\ket{1}\right)$$
5. Sul risultato $1$, quando misuriamo in qubit ausiliario, lo stato è
$$\frac{\frac{1}{\sqrt{2}}\ket{00}\ket{u_1}\frac{1}{2}\ket{1} + \frac{1}{\sqrt{2}}\ket{00}\ket{u_2}\frac{1}{4}\ket{1}}{\sqrt{\frac{5}{32}}}$$
E possiamo notare che $$\frac{\frac{1}{2\sqrt{2}}\ket{u_1} + \frac{1}{4\sqrt{2}}\ket{u_2}}{\sqrt{\frac{5}{32}}} = \frac{\ket{x}}{\|x\|}$$
6. Senza usare altri gate, possiamo calcolare la norma di $\ket{x}$: è la probabilità di misurare $1$ nel bit ausiliario dal precedente step
$$P(\ket{1}) = \left(\frac{1}{2\sqrt{2}}\right)^2 + \left(\frac{2}{4\sqrt{2}}\right)^2 = \frac{5}{32}

##### Implementazione in Qiskit

Qiskit fornisce già un'implementazione dell'algoritmo HHL che richiede solo la matrice $A$ e $\ket{b}$ come input. Possiamo dare all'algoritmo una matrice Hermitiana generica e uno stato iniziale arbitrario come array NumPy, ma non otterrà uno speedup esponenziale perché l'implementazione di default è esatta e quindi esponenziale nel numero di qubit (non c'è nessuno algoritmo in grado di preparare esattamente un stato quanti arbitrario usando risorse polinomiali nel numero di qubit, o che può eseguire esattamente l'operazione $e^{iAt}$ per qualche matrice Hermitiana generica $A$ usando risorse polinomiali nel numero di qubit). Se si conoscono implementazioni efficienti per un particolare problema, la matrice e/o il vettore possono essere forniti come oggetti `QuantumCircuit`.

L'interfaccia per gli algoritmi che risolvono sistemi lineari è `LinearSolver`, ed il problema viene specificato quando si chiama il metodo `solve()`.

L'implementazione più semplice prende la matrice e il vettore come array NumPy, e di seguito creiamo anche un `NumPyLinearSolver` (l'algoritmo classico) per validare le nostre soluzioni.

In [3]:
import numpy as np
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
from qiskit.algorithms.linear_solvers.hhl import HHL


A = np.array([[1,-1/3], [-1/3, 1]])
b = np.array([1, 0])
naive_hhl_sol = HHL().solve(A, b)

Per il risolutore classico dobbiamo riscalare il vettore (cioè `b / np.linal.norm(vector)`) per considerare la rinormalizzazione che avviene quando `b` è codificato in uno stato quantico in HHL

In [4]:
classical_sol = NumPyLinearSolver().solve(A, b/np.linalg.norm(b))

Il package `linear_solvers` contiene una cartella chiamata `matrices` che funge da placeholder per implementazioni efficienti di tipologie particolari di matrici. Allo stato attuale contiene solo l'implementazione efficiente (cioè polinomiale nel numero di qubit) di `TridiagonalToeplitz`: matrici reali simmetriche nella forma
$$A=\left(\begin{array}{c c c c}a&b&0&0\\ b&a&b&0\\ 0&b&a&b\\ 0&0&b&a \end{array}\right)$$
$$a,b \in \mathbb{R}$$

Non si considerano matrici non simmetriche poiché HHL assume matrici Hermitiane, cioè $A = \overline{A^T}$

Dato che la matrice dell'esempio è in questa forma possiamo usare un'istanza di `TridiagonalToeplitz(n_qubits, a, b)` e comparare i risultati rispetto al risolvere il sistema con un array come input.

In [5]:
from qiskit.algorithms.linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz


tridi_A = TridiagonalToeplitz(1, 1, -1/3)
tridi_sol = HHL().solve(tridi_A, b)

Ricordiamo che HHL trova una soluzione esponenzialmente più veloce nella dimensione del sistema rispetto alla controparte classica, ma il prezzo da pagare è che non si ottiene una soluzione esatta ma uno stato quantico che rappresenta il vettore $x$, e apprendere tutte le componenti di questo vettore richiederebbe un tempo lineare nella dimensione, diminuendo lo speedup guadagnato.

Quindi possiamo solo calcolare funzioni di $x$ (i così chiamati **observables**) per apprendere informazioni sulla soluzione. Questo è riflesso in `LinearSolverResult` ritornato da `solve()`, che contiene le seguenti proprietà:
- `state`: il circuito che prepara la soluzione o la soluzione come vettore
- `euclidean_norm`: la norma Euclidea se l'algoritmo sa come calcolarla
- `observable`: la lista degli observable calcolati
- `circuit_results`: i risultati osservabili delle lista dei circuiti

Vedimo le soluzioni ottenute. `classical_solution` è il risultato dell'algoritmo classico, quindi chiamando `state` otteniamo un array:

In [6]:
print('Classical state:', classical_sol.state)

Classical state: [1.125 0.375]


Gli altri due esempi erano algoritmi quantici, quindi possiamo solo vedere lo stato quantico che possiamo vedere attraverso il circuito quantico che prepara lo stato soluzione:

In [7]:
print("Naive state")
print(naive_hhl_sol.state)

Naive state
       ┌─────────────┐┌──────┐        ┌─────────┐
  q52: ┤ circuit-537 ├┤3     ├────────┤3        ├
       └─────────────┘│      │┌──────┐│         │
q53_0: ───────────────┤0     ├┤2     ├┤0        ├
                      │  QPE ││      ││  QPE_dg │
q53_1: ───────────────┤1     ├┤1     ├┤1        ├
                      │      ││  1/x ││         │
q53_2: ───────────────┤2     ├┤0     ├┤2        ├
                      └──────┘│      │└─────────┘
  q54: ───────────────────────┤3     ├───────────
                              └──────┘           


In [8]:
print("Tridiagonal state")
print(tridi_sol.state)

Tridiagonal state
       ┌─────────────┐┌──────┐        ┌─────────┐
  q76: ┤ circuit-763 ├┤3     ├────────┤3        ├
       └─────────────┘│      │┌──────┐│         │
q77_0: ───────────────┤0     ├┤2     ├┤0        ├
                      │  QPE ││      ││  QPE_dg │
q77_1: ───────────────┤1     ├┤1     ├┤1        ├
                      │      ││  1/x ││         │
q77_2: ───────────────┤2     ├┤0     ├┤2        ├
                      └──────┘│      │└─────────┘
  q78: ───────────────────────┤3     ├───────────
                              └──────┘           


Ricordando che la norma Euclidea di un vettore $x=(x_1,\ldots,x_N)$ è definita come $\|x\| = \sqrt{\sum_{i=1}^N x_i^2}$, la probabilità di misurare $1$ nel qubit ausiliario nello step 5. è la norma al quadrato di $x$. Questo significa che l'algoritmo HHL può sempre calcolare la norma euclidea della soluzione e possiamo comprare l'accuratezza dei risultati:

In [9]:
print("Classical Euclidean norm:", classical_sol.euclidean_norm)
print("Naive Euclidean norm:", naive_hhl_sol.euclidean_norm)
print("Tridiagonal Euclidean norm:", tridi_sol.euclidean_norm)

Classical Euclidean norm: 1.1858541225631423
Naive Euclidean norm: 1.1858541225631407
Tridiagonal Euclidean norm: 1.185854122563139


Comparare i vettori soluzioni componente per componente è più compless, di nuovo riflettendo l'idea che non possiamo ottenere il vettore soluzione completo dall'algoritmo quantico. Per didattica, possiamo verificare che effettivamente i diversi vettori soluzione ottenuti sono buone approssimazioni anche a livello di componente.

Per fare ciò, prima dobbiamo usare `StateVector` dal package `quantum_info` e estrarre i componenti del vettore di destra, cioè quelli corrispondenti al qubit ancillare (in basso nei circuiti) con valore $1$ e ai qubit di lavoro (i due nel mezzo nei circuiti) con valore $0$. Quindi siamo interessati agli stati $1000$ e $1001$, corrispondneti rispettivamente al primo e secondo componente del vettore soluzione.

In [11]:
from qiskit.quantum_info import Statevector


naive_sv = Statevector(naive_hhl_sol.state).data
tridi_sv = Statevector(tridi_sol.state).data

# Extract the right component: 100 -> index 8, 1001 -> index 9
naive_fullvec = np.array([naive_sv[8], naive_sv[9]])
tridi_fullvec = np.array([tridi_sv[8], tridi_sv[9]])

print("Naive raw solution vector:", naive_fullvec)
print("Tridi raw solution vector:", tridi_fullvec)

Naive raw solution vector: [ 5.74337504e-17+8.93013426e-18j -1.12425450e-17+5.23599292e-16j]
Tridi raw solution vector: [-1.92535646e-16+9.81307787e-17j -2.24504108e-17+6.80168960e-16j]


I componenti immaginari potrebbero far pensare che i risultati siano sbagliati ma essendo molto molto piccoli sono probabilmente dovuti alla precisione del computer, possono essere ignorati:

In [12]:
naive_fullvec = np.real(naive_fullvec)
tridi_fullvec = np.real(tridi_fullvec)

Successivamente si dividono i vettore per le rispettive norme, così da eliminare le costanti provenienti dalle differenti parti del circuito. I vettori soluzione completi possono essere recuperati moltiplicando questi vettori normalizzati per le rispettive norme euclidee calcolate prima.

In [13]:
print('full naive solution vector:', naive_hhl_sol.euclidean_norm*naive_fullvec/np.linalg.norm(naive_fullvec))
print('full tridi solution vector:', tridi_sol.euclidean_norm*tridi_fullvec/np.linalg.norm(tridi_fullvec))
print('classical state:', classical_sol.state)

full naive solution vector: [ 1.16376749 -0.22780523]
full tridi solution vector: [-1.17787369 -0.13734469]
classical state: [1.125 0.375]
